In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv("/data/movielens/movies.csv")

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings = pd.read_csv("/data/movielens/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [14]:
import numpy as np
pd.options.display.max_rows = 10
pd.options.display.max_columns = 1000

In [16]:
(ratings
.groupby("movieId")
.rating
.agg([np.mean, len])
.query("len>100")
 .sort_values("mean", ascending = False)
 .merge(movies, left_index = True, right_on = "movieId")
 [["title", "movieId", "mean"]]
 .iloc[:10, :]
)

,title,movieId,mean
695,"Godfather, The (1972)",858,4.487500
284,"Shawshank Redemption, The (1994)",318,4.487138
977,"Godfather: Part II, The (1974)",1221,4.385185
48,"Usual Suspects, The (1995)",50,4.370647
472,Schindler's List (1993),527,4.303279
951,One Flew Over the Cuckoo's Nest (1975),1193,4.256944
535,Fargo (1996),608,4.256696
266,Pulp Fiction (1994),296,4.256173
2288,American Beauty (1999),2858,4.236364
6916,"Dark Knight, The (2008)",58559,4.235537


In [19]:
movies.iloc[100:105, :2]

,movieId,title
100,110,Braveheart (1995)
101,111,Taxi Driver (1976)
102,112,Rumble in the Bronx (Hont faan kui) (1995)
103,113,Before and After (1996)
104,114,Margaret's Museum (1995)


In [20]:
import sys, glob, os
SPARK_HOME = "/Users/abulbasar/Downloads/spark-2.3.1-bin-hadoop2.7"
sys.path.append(SPARK_HOME + "/python")
sys.path.append(glob.glob(SPARK_HOME + "/python/lib/py4j*.zip")[0])


http://abuls-mbp-5:4041


In [ ]:
from pyspark.sql import SparkSession, functions as F
spark = (SparkSession
         .builder
         .config("spark.master", "local[*]")
         .config("spark.driver.memory", "4G")
         .config("spark.sql.shuffle.partitions", 16)
         .getOrCreate())
sc = spark.sparkContext
print(sc.uiWebUrl)
sql = spark.sql

In [38]:
movies = (spark.read
          .option("header", True)
          .option("inferSchema", True)
          .format("csv")
          .load("/data/movielens/movies.csv"))
ratings = (spark.read
           .option("header", True)
           .option("inferSchema", True)
          .format("csv")
          .load("/data/movielens/ratings.csv")
           .withColumn("timestamp", F.expr("from_unixtime(timestamp)"))
          )

In [39]:
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [40]:
movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [41]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: string (nullable = true)



In [42]:
ratings.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|     31|   2.5|2009-12-14 08:22:24|
|     1|   1029|   3.0|2009-12-14 08:22:59|
|     1|   1061|   3.0|2009-12-14 08:23:02|
|     1|   1129|   2.0|2009-12-14 08:23:05|
|     1|   1172|   4.0|2009-12-14 08:23:25|
|     1|   1263|   2.0|2009-12-14 08:22:31|
|     1|   1287|   2.0|2009-12-14 08:23:07|
|     1|   1293|   2.0|2009-12-14 08:22:28|
|     1|   1339|   3.5|2009-12-14 08:22:05|
|     1|   1343|   2.0|2009-12-14 08:22:11|
|     1|   1371|   2.5|2009-12-14 08:22:15|
|     1|   1405|   1.0|2009-12-14 08:23:23|
|     1|   1953|   4.0|2009-12-14 08:23:11|
|     1|   2105|   4.0|2009-12-14 08:22:19|
|     1|   2150|   3.0|2009-12-14 08:23:14|
|     1|   2193|   2.0|2009-12-14 08:23:18|
|     1|   2294|   2.0|2009-12-14 08:21:48|
|     1|   2455|   2.5|2009-12-14 08:21:53|
|     1|   2968|   1.0|2009-12-14 08:23:20|
|     1|   3671|   3.0|2009-12-1

In [22]:
type(movies)

pyspark.sql.dataframe.DataFrame

In [25]:
import pyspark.sql.functions as F

In [53]:
top10 = (ratings
 .groupBy("movieId")
 .agg(F.avg("rating").alias("avg_rating")
    , F.count("*").alias("rating_count")
    )
 .filter("rating_count>100")
 .join(movies, on = "movieId")
 .select("avg_rating", "title", "movieId")
 .orderBy(F.desc("avg_rating"))
 .limit(10)
)

top10.show(10, False)

+-----------------+--------------------------------------+-------+
|avg_rating       |title                                 |movieId|
+-----------------+--------------------------------------+-------+
|4.4875           |Godfather, The (1972)                 |858    |
|4.487138263665595|Shawshank Redemption, The (1994)      |318    |
|4.385185185185185|Godfather: Part II, The (1974)        |1221   |
|4.370646766169155|Usual Suspects, The (1995)            |50     |
|4.30327868852459 |Schindler's List (1993)               |527    |
|4.256944444444445|One Flew Over the Cuckoo's Nest (1975)|1193   |
|4.256696428571429|Fargo (1996)                          |608    |
|4.256172839506172|Pulp Fiction (1994)                   |296    |
|4.236363636363636|American Beauty (1999)                |2858   |
|4.235537190082645|Dark Knight, The (2008)               |58559  |
+-----------------+--------------------------------------+-------+



In [57]:
ratings.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|     31|   2.5|2009-12-14 08:22:24|
|     1|   1029|   3.0|2009-12-14 08:22:59|
|     1|   1061|   3.0|2009-12-14 08:23:02|
|     1|   1129|   2.0|2009-12-14 08:23:05|
|     1|   1172|   4.0|2009-12-14 08:23:25|
|     1|   1263|   2.0|2009-12-14 08:22:31|
|     1|   1287|   2.0|2009-12-14 08:23:07|
|     1|   1293|   2.0|2009-12-14 08:22:28|
|     1|   1339|   3.5|2009-12-14 08:22:05|
|     1|   1343|   2.0|2009-12-14 08:22:11|
|     1|   1371|   2.5|2009-12-14 08:22:15|
|     1|   1405|   1.0|2009-12-14 08:23:23|
|     1|   1953|   4.0|2009-12-14 08:23:11|
|     1|   2105|   4.0|2009-12-14 08:22:19|
|     1|   2150|   3.0|2009-12-14 08:23:14|
|     1|   2193|   2.0|2009-12-14 08:23:18|
|     1|   2294|   2.0|2009-12-14 08:21:48|
|     1|   2455|   2.5|2009-12-14 08:21:53|
|     1|   2968|   1.0|2009-12-14 08:23:20|
|     1|   3671|   3.0|2009-12-1

In [55]:
from pyspark.ml.recommendation import ALS

In [56]:
als = ALS(rank = 10, maxIter=10, userCol="userId", seed = 1, 
        , itemCol="movieId", ratingCol="rating" )

In [59]:
df_train, df_test = ratings.randomSplit([0.7, 0.3], seed=1)

In [60]:
df_train.cache()

DataFrame[userId: int, movieId: int, rating: double, timestamp: string]

In [62]:
als_model = als.fit(df_train)

In [63]:
als_model.itemFactors

DataFrame[id: int, features: array<float>]

In [64]:
als_model.itemFactors.select("id").distinct().count()

8033

In [66]:
df_train.selectExpr("count(distinct movieId)").show()

+-----------------------+
|count(DISTINCT movieId)|
+-----------------------+
|                   8033|
+-----------------------+



In [69]:
df_test_pred = als_model.transform(df_test)

In [71]:
df_test.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|   1172|   4.0|2009-12-14 08:23:25|
|     1|   1371|   2.5|2009-12-14 08:22:15|
|     1|   2294|   2.0|2009-12-14 08:21:48|
|     1|   2968|   1.0|2009-12-14 08:23:20|
|     2|     10|   4.0|1996-06-21 16:41:33|
|     2|     50|   4.0|1996-06-21 16:43:06|
|     2|    144|   3.0|1996-06-21 16:50:16|
|     2|    165|   3.0|1996-06-21 16:40:41|
|     2|    208|   3.0|1996-06-21 16:41:51|
|     2|    223|   1.0|1996-06-21 16:45:49|
|     2|    225|   3.0|1996-06-21 16:42:32|
|     2|    235|   3.0|1996-06-21 16:44:24|
|     2|    253|   4.0|1996-06-21 16:41:51|
|     2|    261|   4.0|1996-06-21 16:44:41|
|     2|    292|   3.0|1996-06-21 16:41:32|
|     2|    317|   2.0|1996-06-21 16:42:31|
|     2|    350|   4.0|1996-06-21 16:44:57|
|     2|    367|   3.0|1996-06-21 16:43:39|
|     2|    537|   4.0|1996-06-21 16:53:19|
|     2|    589|   5.0|1996-06-2

In [70]:
df_test_pred.show()

+------+-------+------+-------------------+----------+
|userId|movieId|rating|          timestamp|prediction|
+------+-------+------+-------------------+----------+
|   157|     12|   2.0|2010-12-06 06:46:04|  2.171139|
|   564|     12|   1.0|2000-11-20 14:13:41|  4.076747|
|    91|     12|   3.5|2015-11-29 21:48:07| 3.3061037|
|   624|     12|   2.0|2002-04-23 17:39:13| 2.6224556|
|   636|     18|   3.0|1997-02-06 16:39:24|  3.986588|
|   616|     18|   4.0|1997-04-09 13:35:32| 2.7516918|
|   571|     18|   5.0|2012-04-14 00:10:36| 3.7465937|
|   135|     18|   4.0|1996-10-11 06:38:49|  3.416606|
|   255|     18|   2.5|2009-03-14 03:12:02| 3.4502122|
|   461|     18|   1.5|2004-08-08 15:41:27| 1.9044958|
|   408|     18|   5.0|1999-07-28 04:26:50| 4.0670547|
|   507|     18|   4.0|1997-04-27 03:27:19| 2.9539175|
|   177|     18|   4.0|1998-10-03 07:46:34| 3.5733333|
|    30|     18|   2.0|1999-12-15 22:42:51|  3.209936|
|   655|     18|   4.0|2016-08-01 23:13:09|  3.227679|
|   165|  

In [73]:
df_test_pred.selectExpr("avg(pow(prediction - rating, 2))").show()

+--------------------------------------------------------------------+
|avg(POWER((CAST(prediction AS DOUBLE) - rating), CAST(2 AS DOUBLE)))|
+--------------------------------------------------------------------+
|                                                                 NaN|
+--------------------------------------------------------------------+



In [74]:
df_test_pred.filter("isnan(prediction)").show()

+------+-------+------+-------------------+----------+
|userId|movieId|rating|          timestamp|prediction|
+------+-------+------+-------------------+----------+
|   177|    600|   4.0|1998-10-03 07:47:05|       NaN|
|   232|    961|   5.0|2000-04-07 13:20:25|       NaN|
|   564|    980|   1.0|2000-11-22 02:09:03|       NaN|
|   102|   1384|   4.0|2000-04-24 23:40:21|       NaN|
|   165|   1447|   3.5|2005-03-23 07:53:25|       NaN|
|   609|   1447|   1.0|2002-08-21 00:31:59|       NaN|
|   285|   1447|   4.0|2000-08-06 11:47:34|       NaN|
|   502|   1454|   3.0|1997-04-18 00:24:38|       NaN|
|   581|   2191|   0.5|2009-12-02 09:24:33|       NaN|
|   373|   2246|   2.0|1999-10-04 17:13:10|       NaN|
|   547|   2260|   2.0|2001-02-05 00:41:26|       NaN|
|   564|   2260|   2.0|2000-11-22 02:14:41|       NaN|
|    33|   2507|   1.0|2002-09-23 13:04:23|       NaN|
|   346|   2625|   4.0|2003-02-08 02:30:33|       NaN|
|   195|   3122|   3.0|2000-12-25 18:54:21|       NaN|
|   564|  

In [75]:
df_train.filter("userId = 177").show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|   177|      1|   5.0|1998-10-04 21:11:19|
|   177|     22|   4.0|1998-10-03 07:43:01|
|   177|     23|   3.0|1998-10-03 07:43:01|
|   177|     32|   4.0|1998-10-03 07:30:00|
|   177|     70|   4.0|1998-10-03 07:42:25|
|   177|     89|   4.0|1998-10-03 07:46:34|
|   177|    107|   3.0|1998-10-03 07:46:04|
|   177|    111|   4.0|1998-10-03 07:41:50|
|   177|    160|   2.0|1998-10-03 07:36:09|
|   177|    161|   4.0|1998-10-03 07:43:01|
|   177|    163|   4.0|1998-10-03 07:46:04|
|   177|    164|   4.0|1998-10-03 07:46:04|
|   177|    165|   5.0|1998-10-03 07:45:18|
|   177|    170|   4.0|1998-10-03 07:47:05|
|   177|    172|   4.0|1998-10-03 07:35:21|
|   177|    173|   3.0|1998-10-03 07:30:56|
|   177|    185|   4.0|1998-10-03 07:32:18|
|   177|    196|   3.0|1998-10-03 07:30:56|
|   177|    200|   4.0|1998-10-03 07:51:46|
|   177|    223|   5.0|1998-10-0

In [76]:
df_train.filter("movieId = 600").show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
+------+-------+------+---------+



In [77]:
df_test_pred.filter("not isnan(prediction)")\
.selectExpr("avg(pow(prediction - rating, 2))").show()

+--------------------------------------------------------------------+
|avg(POWER((CAST(prediction AS DOUBLE) - rating), CAST(2 AS DOUBLE)))|
+--------------------------------------------------------------------+
|                                                  0.8702474021627472|
+--------------------------------------------------------------------+



In [79]:
als = ALS(rank = 20, maxIter=10, userCol="userId", seed = 1
        , itemCol="movieId", ratingCol="rating" )
als_model = als.fit(df_train)
df_test_pred = als_model.transform(df_test)
df_test_pred.filter("not isnan(prediction)")\
.selectExpr("avg(pow(prediction - rating, 2))").show()

+--------------------------------------------------------------------+
|avg(POWER((CAST(prediction AS DOUBLE) - rating), CAST(2 AS DOUBLE)))|
+--------------------------------------------------------------------+
|                                                  0.8673324594599199|
+--------------------------------------------------------------------+



In [82]:
als = ALS(rank = 50, maxIter=20, userCol="userId", seed = 1
        , itemCol="movieId", ratingCol="rating" )
als_model = als.fit(df_train)
df_test_pred = als_model.transform(df_test)
df_test_pred.filter("not isnan(prediction)")\
.selectExpr("sqrt(avg(pow(prediction - rating, 2))) rmse").show()

+-----------------+
|             rmse|
+-----------------+
|0.924946783799751|
+-----------------+



In [85]:
reco_10 = als_model\
.recommendForUserSubset(df_test.filter("userId = 100"), 10)

In [84]:
df_test.filter("userId = 100").show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|   100|     52|   3.0|1997-01-25 17:37:36|
|   100|     88|   2.0|1997-01-25 17:40:08|
|   100|     95|   3.0|1997-01-25 17:36:17|
|   100|    608|   4.0|1997-01-25 17:37:04|
|   100|    648|   3.0|1997-01-25 17:36:17|
|   100|    736|   3.0|1997-01-25 17:36:17|
+------+-------+------+-------------------+



In [94]:
(reco_10
.withColumn("recommendation", F.explode("recommendations"))
 .withColumn("movieId", F.expr("recommendation.movieId"))
 .withColumn("rating_reco", F.expr("recommendation.rating"))
 .join(movies, on = "movieId")
 .select("rating_reco", "title")
).toPandas()

,rating_reco,title
0,4.604413,"Goat, The (1921)"
1,4.604413,Land of Silence and Darkness (Land des Schweig...
2,4.604413,Cops (1922)
3,4.604413,"Play House, The (1921)"
4,4.432186,Dog Day Afternoon (1975)
5,4.371039,The Jinx: The Life and Deaths of Robert Durst ...
6,4.334894,Tom Jones (1963)
7,4.325214,"Prophet, A (Un Prophète) (2009)"
8,4.314913,Hotel Rwanda (2004)
9,4.290957,"Producers, The (1968)"
